In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.mini_jasper as mini_jasper
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [5]:
config = malaya_speech.config.ctc_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(**config)

In [7]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v, length_v

(<tf.Tensor 'ExpandDims_4:0' shape=(1, ?, 80) dtype=float32>,
 <tf.Tensor 'ExpandDims_3:0' shape=(1,) dtype=int32>)

In [8]:
model = mini_jasper.Model(v, length_v)
model.logits


Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


{'outputs': <tf.Tensor 'w2l_encoder/Relu_32:0' shape=(1, ?, 1024) dtype=float32>,
 'src_length': <tf.Tensor 'w2l_encoder/floordiv_64:0' shape=(1,) dtype=int32>}

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [11]:
l = sess.run(model.logits, feed_dict = {i: y})
l

{'outputs': array([[[0.01380536, 0.        , 0.        , ..., 0.04834894,
          0.13202894, 0.15845054],
         [0.05161336, 0.        , 0.26168475, ..., 0.6919311 ,
          0.24750529, 0.5443673 ],
         [0.09645983, 0.18899249, 0.11485588, ..., 0.95225793,
          0.30504438, 0.        ],
         ...,
         [0.8125198 , 0.        , 0.62700826, ..., 0.41779   ,
          0.15089075, 0.7497936 ],
         [0.18092753, 0.26628187, 0.        , ..., 0.08859082,
          0.        , 0.26862836],
         [0.28583166, 0.        , 0.06555645, ..., 0.        ,
          0.        , 0.        ]]], dtype=float32),
 'src_length': array([281], dtype=int32)}

In [12]:
l = sess.run(model.logits, feed_dict = {i: y1})
l

{'outputs': array([[[0.63250047, 0.        , 0.        , ..., 0.4929435 ,
          0.        , 0.        ],
         [1.1794008 , 0.        , 0.        , ..., 0.        ,
          0.        , 0.6498845 ],
         [0.        , 0.23245783, 0.41061568, ..., 0.3460966 ,
          0.        , 0.6457398 ],
         ...,
         [0.7224293 , 0.        , 0.93415856, ..., 0.18927753,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.15736698,
          0.08533047, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.02290356]]], dtype=float32),
 'src_length': array([175], dtype=int32)}